In [ ]:

#Output of code ommited for clarity of script

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn import preprocessing
from sklearn.model_selection import LeavePOut
from sklearn import decomposition
from sklearn import model_selection

from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
pd.set_option('display.max_columns', 100)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


data = pd.read_csv('C:/Users/u0132612/Documents/PhD KULeuven/Lessen/Advanced Analytics in a Big Data World/Assignments/Assignment 2/train.csv')
data_test = pd.read_csv('C:/Users/u0132612/Documents/PhD KULeuven/Lessen/Advanced Analytics in a Big Data World/Assignments/Assignment 2/test.csv')

data.info()


#SAME PREPROCESSING as done by Steven

data.pixels_y = data.pixels_y.astype('int64')

data_test.pixels_y = data_test.pixels_y.fillna(0)
data_test.pixels_y = data_test.pixels_y.astype('int64')

#selecteer enkel usefull information
data_s = data[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', "cpu_details", 'discrete_gpu','gpu','os','os_details','ram','ssd','storage','weight','min_price','max_price']]
data_test_s = data_test[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu', 'cpu_details', 'discrete_gpu','gpu',"os",'os_details','ram','ssd','storage','weight']]

#first basic reformatter
def reformat(df):
    df_pp = df.copy()
    df_pp["cpu_brand"] = df_pp.cpu.str.split(n=1).str[0]
    df_pp["cpu_type"] = df_pp.cpu.str.split(n=1).str[1]
    df_pp["gpu_brand"] = df_pp.gpu.str.split(n=1).str[0]
    df_pp["gpu_series"] = df_pp.gpu.str.split(n=2).str[1]
    df_pp["os_type"] = df_pp.os_details.str.split(n=1).str[1]
    df_pp["cpu_gen"] = df_pp['cpu_details'].str.split("(").str[1].str.split(")").str[0].str.split(" ",n=3).str[0]
    df_pp = df_pp.drop(columns=["os_details", "cpu", "gpu", "os_details"])
    return df_pp

#automatische onehot encoder
def process_onehot(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        enc = preprocessing.OneHotEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        new_df = pd.DataFrame(data=enc.transform(df[[item]].dropna()).toarray(), columns=enc.categories_[0])
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

#automatische ordinal encoder
def process_ordinal(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        new_df = pd.DataFrame()
        enc = preprocessing.OrdinalEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        temp = enc.transform(df[[item]].dropna()).shape
        new_df[item] = enc.transform(df[[item]].dropna()).flatten()
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

data_s = reformat(data_s)
data_test_s = reformat(data_test_s)

data_t = pd.concat([data_s, data_test_s]).reset_index(drop=True)

processed_data_total = process_ordinal(data_t,['screen_surface', 'cpu_type', 'gpu_brand', "gpu_series", 'os_type','brand', "cpu_gen","os" ])

data_t = data_t[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]

data_total = pd.concat([processed_data_total, data_t], axis=1).set_index("id")

#selecteer de juiste datasets -> training voor model training en test de test dataset van seppe

data_total_train = data_total.filter(items=data.id, axis=0)
data_total_test = data_total.filter(items=data_test.id, axis=0).iloc[:,0:-2].fillna(0)
index=data_total_test.index


X_1 = data_total_train.iloc[:,0:-2].fillna(0)
y_min = data_total_train.iloc[:,-2].fillna(0)
X_2 = data_total_train.iloc[:,0:-2].fillna(0)
y_max = data_total_train.iloc[:,-1].fillna(0)

pp = preprocessing.PowerTransformer(standardize=True)

                            #########################
                            ## Build XGBoost model ##

    #######################
    # MINIMUM price model #
    #######################
    
MinDmatrix= xgb.DMatrix(X_1, label=y_min)

X_train, X_test, y_train, y_test = train_test_split(X_1,y_min,test_size=.3, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

from sklearn.metrics import mean_absolute_error

# Calculate a simple baseline based on mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE is {:.2f}".format(mae_baseline))

# Baseline MAE is 448.74

# Develop more advanced model with Tuning

params = {
    # Parameters that we are going to tune.
    'max_depth':6,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    # Other parameters
    'objective':'reg:linear',
}

params['eval_metric'] = "mae"
num_boost_round = 999                  #I will apply early stopping later on

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

# Best MAE: 176.71 with 12 rounds

    # After the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. 
    # This is the MAE of the model with default parameters and an optimal number of boosting rounds, on the test (validation) dataset. 
    # As you can see, better performance than baseline 


#In order to tune the other hyperparameters, I use the cv function from XGBoost. 
#It allows to run cross-validation on the training dataset and returns a mean MAE score.


cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

cv_results['test-mae-mean'].min()

# Start with max depth and min child weight (maybe should do this in a GridSearch that combines all hyperparms)

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,15)
    for min_child_weight in range(1,25)
]

min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

# Best params: 5, 6, MAE: 152.267633
# Did exagerate a bit but best params are max_depth=5 & min_child_weight=6 [still model for min value]

params['max_depth'] = 5
params['min_child_weight'] = 6

# Let's look for subsample and colsample optimal values

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

# Best params: 0.8, 0.7, MAE: 145.8399596
# best params are subsample=0.8 & colsample= 0.7 [still model for min value]

params['subsample'] = .8
params['colsample_bytree'] = 0.7

# Now i look for the optimal learning rate (eta)

min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

# Best params: 0.01, MAE: 143.9732178
# best param for learning rate = 0.01 [still model for min value]

params['eta'] = .01

# Create param dict with the optimal hyperparams

params
{'colsample_bytree': 0.7,
 'eta': 0.01,
 'eval_metric': 'mae',
 'max_depth': 5,
 'min_child_weight': 6,
 'objective': 'reg:linear',
 'subsample': 0.8}

# Train model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1)) 

# Best MAE: 155.62 in 385 rounds

num_boost_round = model.best_iteration + 1

# Before using it for predictions, retrain it with the good number of rounds. 
# Since  the exact best num_boost_round is known, don't need the early_stopping_round anymore.

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

# Make predictions with the model on test data set for MIN PRICE VALUE

mean_absolute_error(best_model.predict(dtest), y_test) # MAE = 155.62180246789472 in 385 rounds


                                    #########################
                                    # Build XGBoost model 2 #
                                    #########################
    #######################
    # MAXIMUM price model #
    
MaxDmatrix= xgb.DMatrix(X_2, label=y_max)

X_train, X_test, y_train, y_test = train_test_split(X_2,y_max,test_size=.3, random_state=42)

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

from sklearn.metrics import mean_absolute_error

# Calculate a simple baseline based on mean from the training data
mean_train = np.mean(y_train)
# Get predictions on the test set
baseline_predictions = np.ones(y_test.shape) * mean_train
# Compute MAE
mae_baseline = mean_absolute_error(y_test, baseline_predictions)
print("Baseline MAE MAX is {:.2f}".format(mae_baseline))

# Baseline MAE MAX is 465.21

# Develop more advanced model with Tuning

params = {
    'max_depth':6,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective':'reg:linear',
}

params['eval_metric'] = "mae"
num_boost_round = 999                  #I will apply early stopping later on

model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE MAX: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))

# Best MAE MAX: 182.83 with 12 rounds
# Again, after the 12th tree, adding more rounds did not lead to improvements of MAE on the test dataset. 
# This is the MAE of the model with default parameters and an optimal number of boosting rounds, on the test (validation) dataset.

# Tune other hyperparams, using cv function from XGBoost. Run cross-val on training and return mean MAE score

cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    seed=42,
    nfold=5,
    metrics={'mae'},
    early_stopping_rounds=10
)
cv_results

cv_results['test-mae-mean'].min()

# Start with max depth and min child weight (maybe should do this in a GridSearch that combines all hyperparms)

gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(2,15)
    for min_child_weight in range(1,25)
]

min_mae = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

# MAE 158.5139102 for 21 rounds
# Best params: 5, 7, MAE: 152.5074432

# Best params are max_depth= 5 & min_child_weight= 7 [still model for Max price]

params['max_depth'] = 5
params['min_child_weight'] = 7

# Let's look for subsample and colsample optimal values

gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(1,11)]
    for colsample in [i/10. for i in range(1,11)]
]

min_mae = float("Inf")
best_params = None
# We start by the largest values and go down to the smallest
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # We update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        seed=42,
        nfold=5,
        metrics={'mae'},
        early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))

# MAE 213.44088739999998 for 82 rounds
# Best params: 0.9, 1.0, MAE: 151.77083720000002

# best params are subsample=0.9 & colsample= 1.0 [still model for MAX value]

params['subsample'] = .9
params['colsample_bytree'] = 1.0

# Now i look for the optimal learning rate (eta)

min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=num_boost_round,
            seed=42,
            nfold=5,
            metrics=['mae'],
            early_stopping_rounds=10
    )
    # Update best score
    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds\n".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = eta
print("Best params: {}, MAE: {}".format(best_params, min_mae))

# Best params: 0.3, MAE: 151.77083720000002
# best param for learning rate = 0.3 (really different from min model, 0.01) [still model for MAX value]

params['eta'] = .3

# Create param dict with the optimal hyperparams

params
{'colsample_bytree': 1.0,
 'eta': 0.3,
 'eval_metric': 'mae',
 'max_depth': 5,
 'min_child_weight': 7,
 'objective': 'reg:linear',
 'subsample': 0.9}

# Train model
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

print("Best MAE MAX: {:.2f} in {} rounds".format(model.best_score, model.best_iteration+1)) 

# Best MAE MAX: 159.17 in 28 rounds (<->Best MAE min was 155.62 in 385! rounds)

num_boost_round = model.best_iteration + 1

# Before using it for predictions, we should retrain it with the good number of rounds. 
#Since we know the exact best num_boost_round, we don't need the early_stopping_round anymore.

best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")]
)

# Make predictions with the model on test data set (VALIDATION SET based on training set)

mean_absolute_error(best_model.predict(dtest), y_test) # MAE max= 159.16917813968035 in 28 rounds (MAE Min = 155.62180246789472 in 385 rounds)

                                        
    ####### Combined result on validation set ######


#total MAE based on created VALIDATION set BUT averaged over the PC ID's
#the test set was not used

MAEmin = 155.62180246789472
MAEmax = 159.16917813968035

totalerror = MAEmin + MAEmax
print(totalerror)

# 314.7909806075751